In [ ]:
!pip install qubovert
!pip install qiskit-aqua
!pip install qiskit

In [149]:
import qiskit
qiskit.__version__

'0.19.1'

In [150]:
from qiskit import BasicAer
from qiskit.aqua import aqua_globals, QuantumInstance

from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import MinimumEigenOptimizer, RecursiveMinimumEigenOptimizer
from qiskit.optimization import QuadraticProgram

import math
import numpy as np
import pandas as pd
from sympy import *
import re
import time
import random
import itertools

In [151]:
def generate_problem_instance(distribution, n_agents):
  if distribution.lower() == 'random' or distribution.lower() == 'r':
    coalition_values = {}
    for i in range(1,n_agents+1):
      temp = list(itertools.combinations(range(1,n_agents+1), i))
      for coalition in temp:
        coalition_values[','.join([str(x) for x in coalition])] = random.randint(1,100)
  if distribution.lower() == 'normal' or distribution.lower() == 'n':
    mu, sigma = 50, 20 # mean and standard deviation
    s = np.random.normal(mu, sigma, 2**n_agents)
    coalition_values = {}
    idx = 0
    for i in range(1,n_agents+1):
      temp = list(itertools.combinations(range(1,n_agents+1), i))
      for coalition in temp:
        coalition_values[','.join([str(x) for x in coalition])] = int(s[idx])
        idx+=1
  if distribution.lower() == 'agent-based-uniform' or distribution.lower() == 'abu':
    '''

    '''
  if distribution.lower() == 'agent-based-normal' or distribution.lower() == 'abn':
    '''
    
    '''
  if distribution.lower() == 'modified-uniform' or distribution.lower() == 'mu':
    '''
    
    '''
  if distribution.lower() == 'rayleigh' or distribution.lower() == 'r':
    '''
    
    '''
  if distribution.lower() == 'single-value-agent' or distribution.lower() == 'sva':
    '''

    '''
  return coalition_values

In [152]:
def convert_to_BILP(coalition_values):
  '''

  '''
  x={}
  for i in range(len(coalition_values)):
    x[i] = symbols(f'x_{i}')
  n = list(coalition_values.keys())[-1].count(',')+1                              #get number of agents
  S=[]
  for agent in range(n):
    temp = []
    for coalition in coalition_values.keys():
      if str(agent+1) in coalition:
        temp.append(1)                                                            #'1' if agent is present in coalition
      else:
        temp.append(0)                                                            #'0' if agent is not present in coalition
    S.append(temp)
  b=[1]*n                                                                         # vector b is a unit vector in case of BILP of CSGP
  c = list(coalition_values.values())                                             # vector of all costs (coalition values)
  return (c,S,b)

In [153]:
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
def atoi(text):
    return int(text) if text.isdigit() else text


def get_QUBO_coeffs(c,S,b,P):
  '''

  '''
  x={}
  for i in range(len(c)):
    x[i] = symbols(f'x_{i}')
  final_eq = simplify(sum([c_value*x[i] for i,c_value in enumerate(c)])+P*sum([expand((sum([x[idx] for idx,element in enumerate(agent) if element])-1)**2) for agent in S])) #simplify numerical equation
  linear={}
  quadratic={}
  for term in final_eq.as_coeff_add()[1]:
    term = str(term)
    if '**' in term:                                                              #get the coefficient of the squared terms as linear coefficients (diagonal elements of the Q matrix in the QUBO problem)
      if term.count('*')==3:
        linear[term.split('*')[1]] = int(term.split('*')[0])
      else:
        if not term.startswith('-'):
          linear[term.split('**')[0]] = int(1)
        else:
          linear[term.split('**')[0][1:]] = int(-1)
    elif term.count('*')==1:                                                        #get the coefficient of the linear terms (diagonal elements of the Q matrix in the QUBO problem)
      linear[term.split('*')[1]] += int(term.split('*')[0])
    else:
      quadratic[(term.split('*')[1],term.split('*')[2])] = int(term.split('*')[0])  #get the coefficient of quadratic terms (upper diagonal elements of the Q matrix of the QUBO problem)  
  return linear,quadratic

In [154]:
def solve_QUBO(linear, quadratic, algo='qaoa', p=1):
  '''
  
  '''
  keys = list(linear.keys())
  keys.sort(key=natural_keys)
  linear = [linear[key] for key in keys]
  qubo = QuadraticProgram()
  for i in range(len(linear)):
    qubo.binary_var(f'x_{i}')                                                             #initialize the binary variables
  qubo.maximize(linear=linear, quadratic=quadratic)                                       #initialize the QUBO maximization problem instance
  op, offset = qubo.to_ising()
  qp=QuadraticProgram()
  qp.from_ising(op, offset, linear=True)
  aqua_globals.random_seed = 123
  quantum_instance = QuantumInstance(BasicAer.get_backend('statevector_simulator'),       #qasm simulator
                                     seed_simulator=aqua_globals.random_seed,
                                     seed_transpiler=aqua_globals.random_seed)
  if algo == 'qaoa':
    qaoa_mes = QAOA(quantum_instance=quantum_instance, initial_point=[0., 0.])
    qaoa = MinimumEigenOptimizer(qaoa_mes)    # using QAOA
    result = qaoa.solve(qubo)
  else:
    exact_mes = NumPyMinimumEigensolver()
    exact = MinimumEigenOptimizer(exact_mes)  # using the exact classical numpy minimum eigen solver
    result = exact.solve(qubo)
  return result

In [155]:
#convert solution binary string to coalition structure and also output the coalition
def decode(solution,coalition_values):
  '''

  '''
  output = []
  for index,element in enumerate(solution):
    if int(element)!=0:
      output.append(set(list(coalition_values)[index].split(',')))
  return output

In [156]:
def get_linear_quads(distribution,n):
  coalition_values = generate_problem_instance(distribution, n)
  c,S,b = convert_to_BILP(coalition_values)
  qubo_penalty =-50
  linear,quadratic = get_QUBO_coeffs(c,S,b,qubo_penalty)
  return coalition_values,linear,quadratic

##Classical Solver

In [157]:
#Classical BILP solver

def constraint(x,S):
  '''
  
  '''
  for i in range(len(S)):
    temp = 0
    for j in range(len(x)):
      temp+= (S[i][j]*int(x[j]))
    if temp!=1:
      return False
  return True


def solve_BILP_classical(coalition_values):
  '''
  
  '''
  S=[]
  for i in range(math.ceil(math.sqrt(len(coalition_values)))):                    #get number of agents
    S.append([])
    for j,value in coalition_values.items():
      if str(i+1) in j:
        S[i].append(1)
      else:
        S[i].append(0)
  optimal_cost = 0
  for b in range(2**len(coalition_values)):
    x = [int(t) for t in reversed(list(bin(b)[2:].zfill(len(coalition_values))))]
    x = ''.join(str(e) for e in x)
    if constraint(x,S):
      cost = 0
      for j in range(len(x)):
        cost+=coalition_values[list(coalition_values.keys())[j]]*int(x[j])
      if optimal_cost<cost:
        optimal_cost = cost
        optimal_x = x
  return optimal_x

##Execute Functions and Generate Table

In [174]:
distributions = ['RANDOM','NORMAL']
n_agents = [2,3]

table_headers = ['Distribution', 'No. of Agents','Solution Function Value','Solution Binary String','QAOA Solution','True Solution (Classical)']
table_contents = []

for distribution in distributions:
  for n in n_agents:
    start_time = time.time()
    print(f'Executing {distribution} distribution for {n} agents',end='')
    row = []
    coalition_values, linear, quadratic = get_linear_quads(distribution,n)
    p = 3                                            # number of QAOA layers
    solution = solve_QUBO(linear,quadratic,'qaoa',p)
    optimal_coalition_structure = decode(solution.x,coalition_values)

    row.append(distribution)
    row.append(n)
    row.append(solution.fval)
    row.append(list(map(int,solution.x)))
    row.append(decode(solution.x,coalition_values))
    print(f', Time taken = {time.time()-start_time}')
    row.append(decode(list(solve_BILP_classical(coalition_values)),coalition_values))     #Calculating classical solution
    table_contents.append(row)

Executing RANDOM distribution for 2 agents, Time taken = 0.4933810234069824
Executing RANDOM distribution for 3 agents, Time taken = 2.982795000076294
Executing NORMAL distribution for 2 agents, Time taken = 0.7419557571411133
Executing NORMAL distribution for 3 agents, Time taken = 3.646982192993164


In [175]:
#view output table

pd.DataFrame(table_contents, columns=table_headers)

,Distribution,No. of Agents,Solution Function Value,Solution Binary String,QAOA Solution,True Solution (Classical)
0,RANDOM,2,179.0,"[1, 1, 0]","[{1}, {2}]","[{1}, {2}]"
1,RANDOM,3,245.0,"[0, 0, 0, 0, 0, 0, 1]","[{3, 1, 2}]","[{3, 1, 2}]"
2,NORMAL,2,190.0,"[1, 1, 0]","[{1}, {2}]","[{1}, {2}]"
3,NORMAL,3,259.0,"[1, 0, 0, 0, 0, 1, 0]","[{1}, {3, 2}]","[{1}, {3, 2}]"


In [6]:
#Classical BILP solver     2,3,4,5(takes a lot of time)

#Classical QUBO solver     2,3,4,5(resource overflow)gith

#QAOA QUBO solver          2,3,4(15 qubits)(resource overflow),5(31 qubits)(resource overflow)